# Denoising Diffusion Probabilistic Model

## Overview
The denoising diffusion probabilistic model (DDPM) can be viewed as a Markovian hierarchical
variational autoencoder, where the forward encoder is a fixed linear Gaussian model, and
we are interested in learning the reverse decoder.
The idea is to progressively add Gaussian noise to the input data in the forward process
until it is indistinguishable from the standard Gaussian noise and learn a denoising model
in the backward process to reconstruct the input from the noise.

Specifically, given the Markov property, the forward process can be factorized as
$q(x_{1:T}|x_0)= \prod_{t=1}^T q(x_t|x_{t-1})$, where $x_0$ is the input and $x_{1:T}$ are
the latent variables given the input. We have
$q(x_t|x_{t-1}) = \mathcal{N}(x_t|\sqrt{\alpha_t} x_{t-1}, (1-\alpha_t)I)$, and
$\alpha_t = 1-\beta_t$ where $\beta_t$ is the noise level added at each layer of the forward
encoder.

The join distribution of the reverse decoder is
$p(x_{0:T})=p(x_T)\prod_{t=1}^T p_\theta(x_{t-1}|x_t)$ where $p(x_T)=\mathcal{N}(x_T|0, I)$.
Our goal is to learn the parameters $\theta$ by maximizing the variational lower bound which
is derived from the forward and reverse processes.

## Method

### Training
Here are the training steps of the model:
1. Given an input data batch $x_0 \sim q(x_0)$, we uniformly sample
 the timestep index for each data in the batch that corresponds to the associated
noisy sample, $t \sim \text{Uniform}(\{1, \dots, T\})$.
We also sample standard Gaussian noise for each data for the noise process,
$\epsilon_0 \sim \mathcal{N}(\epsilon_0|0, I)$.
2. In the forward encoder, we run the noise process $q(x_t|x_{t-1})$ for each sample in
the input batch up to timestep $t$.
Since the noise process is a linear Gaussian model, we can derive the
closed-form formula of the noisy sample based on the input,
initial noise $\epsilon_0$ and timestep $t$:

$$
q(x_t|x_0) \sim \mathcal{N}(x_t|\sqrt{\bar{\alpha}_t} x_0, (1 -\bar{\alpha}_t) I)
$$

where $\bar α_t = \prod_{i=1}^t α_i$. By using the reparameterization trick,
we have:

$$
x_t = \sqrt{\bar{\alpha}_t} x_0 + \sqrt{1 -\bar{\alpha}_t} \epsilon_0
$$

3. In the reverse decoder, our goal is to learn the true conditional
denoising distribution $q(x_{t-1}|x_t, x_0)$, which can be shown to be
a Gaussian distribution
$q(x_{t-1}|x_t, x_0) \sim \mathcal{N}(x_{t-1}|\mu_q(x_t, x_0), \Sigma_q(t))$, where:

$$
\begin{aligned}
\mu_q(x_t, x_0) &= \frac{\sqrt{\alpha_t} (1 - \bar{\alpha}_{t-1}) x_t + \sqrt{\bar{\alpha}_{t-1}} (1 - \alpha_t) x_0}{1 - \bar{\alpha}_t}\\
\Sigma_q(t) &= \sigma_q^2(t)I = \frac{(1-\alpha_t)(1-\bar{\alpha}_{t-1})}{1-\bar{\alpha}_t}I
\end{aligned}
$$
We can also derive the mean of the conditional distribution as below by utilizing the reparameterization trick:

$$
\mu_q(x_t, x_0) = \frac{1}{\sqrt{\alpha_t}}x_t - \frac{1 - \alpha_t}{\sqrt{1-\bar{\alpha}_t}\sqrt{\alpha_t}}\epsilon_0
$$

Since we don't have access to the ground-truth input $x_0$ and the
initial noise $\epsilon_0$ during the reverse process, we define
the decoder's denoising model $p_\theta(x_{t-1}|x_t, t)$ as Gaussian
distribution with the following mean and covariance:

$$
\mu_\theta(x_t, t) = \frac{1}{\sqrt{\alpha_t}}x_t - \frac{1 - \alpha_t}{\sqrt{1-\bar{\alpha}_t}\sqrt{\alpha_t}} \hat{\epsilon}_\theta (x_t, t)
$$

where the noise is parameterized by a neural network with parameters $\theta$.

4. It can be shown that maximizing the variational lower bound boils down
to minimizing the following loss:

$$
\mathcal{L}_{\text{simple}}(θ) =
\mathbb{E}_{t \sim \mathcal{U}[1,T],\, ε_0 \sim \mathcal{N}(0,I),\, x_0 \sim \text{data}} \frac{1}{2\sigma_q^2(t)} \frac{(1-\alpha_t)^2}{(1 - \bar{\alpha}_t)\alpha_t} \left[
\left\| ε_0 - \hat ε_θ(x_t, t) \right\|^2_2
\right]
$$

### Sampling
After the training, to generate new samples, execute the reverse process:
1. Sample a random sample $x_T \sim \mathcal{N}(x_T|0, I)$
2. For each step of the reverse process $t=T, \dots, 1$:
    - If $t > 1$ sample a noise $\epsilon \sim \mathcal{N}(\epsilon|0, I)$, otherwise set $\epsilon=0$
    - Compute the denoised sample $x_{t-1}=\mu_\theta(x_t, t) + \sigma_q(t)\epsilon$
